# Índice Invertido utilizando Metapy

In [1]:
import metapy
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession


In [2]:
sc = SparkContext('local', "news_indexed_topic_model") 
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [3]:
dfNews=spark.read.csv("file:///home/ubuntu/Caso_estudio/datasets/news.csv", inferSchema=True, header=True, encoding="UTF-8")
dfNews.show(10)

+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
| id|id_news|               title|   publication|              author|      date|  year|month| url|             content|
+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|  0|  17283|House Republicans...|New York Times|          Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON  —   C...|
|  1|  17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0|  6.0|null|After the bullet ...|
|  2|  17285|Tyrus Wong, ‘Bamb...|New York Times|        Margalit Fox|2017-01-06|2017.0|  1.0|null|When Walt Disney’...|
|  3|  17286|Among Deaths in 2...|New York Times|    William McDonald|2017-04-10|2017.0|  4.0|null|Death may be the ...|
|  4|  17287|Kim Jong-un Says ...|New York Times|       Choe Sang-Hun|2017-01-02|2017.0|  1.0|null|SEOUL, South Kore...|
|  5|  17288|Sick With a Cold,..

In [4]:
!rm -rf news-idx
idx = metapy.index.make_inverted_index('config.toml')

print(f'Total de noticias: {idx.num_docs()}')
print(f'Cantidad de palabras únicas: {idx.unique_terms()}')
print(f'Promedio de longitud de las noticias: {idx.avg_doc_length()}')

Total de noticias: 183222
Cantidad de palabras únicas: 255530
Promedio de longitud de las noticias: 297.3456726074219


In [5]:
dfIndexNews=dfNews.toPandas()

In [6]:
ranker = metapy.index.OkapiBM25(k1=1.2, b=0.75)
query = metapy.index.Document()
query.content('commercial battle against china') 
top_docs = ranker.score(idx, query, num_results=5)

print(top_docs)
index=[tup[0] for tup in top_docs]
dfIndexNews.loc[index,['title','content']]

[(20543, 12.039117813110352), (30497, 11.377471923828125), (152595, 11.307750701904297), (155592, 11.213479042053223), (92512, 11.188894271850586)]


,title,content
20543,WATCH: Chinese Commercial Advertises Detergent...,A new television commercial has been called “ ...
30497,FARAGE FOR BREITBART: Trump Stole the March on...,I can now see where Hillary gets it from. Her ...
152595,"Aleppo fighting rages as U.S., Russia try to r...",Syrian government forces and their allies fou...
155592,"Tech, bank shares slip after recent gains; hea...",The S&P 500 and Nasdaq Composite fell on Mond...
92512,Conservatives Yearning for the 1950s,None
